In [1]:
import zstandard
from alchemiscale import AlchemiscaleClient, Scope
import os

In [2]:
user_id = os.environ['ALCHEMISCALE_ID']
user_key = os.environ['ALCHEMISCALE_KEY']


In [3]:
asc = AlchemiscaleClient(
    'https://api.alchemiscale.org',
    user_id,
    user_key
)


In [4]:
asc.list_scopes()

[<Scope('openff-*-*')>]

In [6]:
import openfe

# Get the alchemical network
alchemical_network = openfe.AlchemicalNetwork.from_json("alchemical_network.json")
print(alchemical_network)


<AlchemicalNetwork-330f4422697b3f1035915299495a0ca0>


/home/joshmitchell/micromamba/envs/alchemiscale-client/lib/python3.12/site-packages/gufe/components/explicitmoleculecomponent.py:93: UserWarning: Partial charges have been provided, these will preferentially be used instead of generating new partial charges
  warnings.warn(wmsg)


In [8]:
# Set the scope for the transformation
scope = Scope("openff", "sage_2_0_0", "problematic_phosphates")
print(f"Scope is set to: {scope}")


Scope is set to: openff-sage_2_0_0-problematic_phosphates


In [9]:
# Create a network and get a scope key
an_sk = asc.create_network(alchemical_network, scope)


Output()

In [11]:
# store the scoped key
with open("scopekey", 'w') as f:
    f.write(str(an_sk))


In [11]:
# When a compute node is ready for a task, it first stochastically chooses a network, then stochastically chooses a task within that network
# The weight is the likelihood that that network/task will be chosen
# Choosing a small value allows other networks to take priority on shared compute nodes.
asc.set_network_weight(an_sk, 0.0001)

<ScopedKey('AlchemicalNetwork-330f4422697b3f1035915299495a0ca0-openff-sage_2_0_0-problematic_phosphates')>

In [17]:
# action out tasks
repeats = 3
print(f"Actioning {repeats} repeats for {len(alchemical_network.edges)} transformation")
for transform_sk in asc.get_network_transformations(an_sk):
    tasks = asc.create_tasks(transform_sk, count=repeats) # will always create a new task
    asc.action_tasks(tasks, an_sk) # idempotent

Actioning 3 repeats for 9 transformation


In [53]:
# check what the network status looks like
asc.get_network_status(an_sk)

AlchemicalNetwork-330f4422697b3f1035915299495a0ca0-openff-sage_2_0_0-problematic_phosphates                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ status                                                           ┃                                        count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ complete                                                         │                                           14 │
│ running                                                          │                                           40 │
│ waiting                                                          │                                            0 │
│ error                                                            │                                            0 │
│ invalid                                                          │                                            0 │
│ deleted                                                          │                                            0 │
└──────────────────────────────────────────────────────────────────┴──────────────────────────────────────────────┘

{'running': 40, 'complete': 14}

In [14]:
asc.get_scope_status() # Show all tasks I have access to (in the openff-*-* task)

*-*-*                                                                                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ status                                                           ┃                                        count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ complete                                                         │                                            0 │
│ running                                                          │                                           27 │
│ waiting                                                          │                                            0 │
│ error                                                            │                                            0 │
│ invalid                                                          │                                            0 │
│ deleted                                                          │                                            0 │
└──────────────────────────────────────────────────────────────────┴──────────────────────────────────────────────┘

{'running': 27}

In [16]:
asc.get_scope_status(Scope("openff"))

openff-*-*                                                                                                         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ status                                                           ┃                                        count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ complete                                                         │                                            0 │
│ running                                                          │                                           27 │
│ waiting                                                          │                                            0 │
│ error                                                            │                                            0 │
│ invalid                                                          │                                            0 │
│ deleted                                                          │                                            0 │
└──────────────────────────────────────────────────────────────────┴──────────────────────────────────────────────┘

{'running': 27}

In [ ]:
# # cancel the tasks
# tasks = asc.get_network_tasks(an_sk)
# asc.cancel_tasks(tasks, an_sk)

In [13]:
# Define/read a deterministic ordering of tasks within this network
# 1 means first

tasks = asc.get_network_tasks(an_sk)
asc.get_tasks_priority(tasks)

[10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10]

In [14]:
asc.get_network_actioned_tasks(an_sk, task_weights=True)

{<ScopedKey('Task-05e37b7366eb4535ac37d16a881fa0cb-openff-sage_2_0_0-problematic_phosphates')>: 0.5,
 <ScopedKey('Task-c444bf27c3784813907dfff1b67f2e59-openff-sage_2_0_0-problematic_phosphates')>: 0.5,
 <ScopedKey('Task-cd30f9358b97476ab4c2a48ed59f89fe-openff-sage_2_0_0-problematic_phosphates')>: 0.5,
 <ScopedKey('Task-0c6881ead72b4e70afacb085276ef5ea-openff-sage_2_0_0-problematic_phosphates')>: 0.5,
 <ScopedKey('Task-4b57ef7e41774b9ca6f07cac7e8d1b3d-openff-sage_2_0_0-problematic_phosphates')>: 0.5,
 <ScopedKey('Task-e07c1dbc884f4e628c81aa9bf68ad3b3-openff-sage_2_0_0-problematic_phosphates')>: 0.5,
 <ScopedKey('Task-fd39dff7d2da49f0a1ee0a2db776d65c-openff-sage_2_0_0-problematic_phosphates')>: 0.5,
 <ScopedKey('Task-61a34c474159498c953a5cc0fac50a4a-openff-sage_2_0_0-problematic_phosphates')>: 0.5,
 <ScopedKey('Task-40b27f4a3b224acbaf69c1de93fe5821-openff-sage_2_0_0-problematic_phosphates')>: 0.5,
 <ScopedKey('Task-6ed044859eea4effaa599bfda6767367-openff-sage_2_0_0-problematic_phosphates

In [6]:
an_sk = asc.query_networks()[0]

In [54]:
results = asc.get_network_results(an_sk)

Output()


********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
               OR                
      conda install pymbar       
*********************************

********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
               OR                
      conda install pymbar       
*********************************

********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
               OR                
      conda install pymbar       
*********************************

********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
          

In [27]:
transformation, result = [(transformation, result) for transformation, result in results.items() if result is not None][0]

In [28]:
result.get_estimate()

-9.737406260846342 <Unit('kilocalorie_per_mole')>

In [32]:
transformation.

'openff'

In [33]:
results

{<ScopedKey('Transformation-32def6039be0d0fcbd89756502244afc-openff-sage_2_0_0-problematic_phosphates')>: None,
 <ScopedKey('Transformation-7e0b538b86fdb78265a8ad6bfb4a324a-openff-sage_2_0_0-problematic_phosphates')>: None,
 <ScopedKey('Transformation-b9c18a2bf9dd329f4a615e7461281a39-openff-sage_2_0_0-problematic_phosphates')>: None,
 <ScopedKey('Transformation-9677044530cf0fda0e0a4b82256bbb89-openff-sage_2_0_0-problematic_phosphates')>: None,
 <ScopedKey('Transformation-747fb61d90c2a3c4f5005787d31d0dbe-openff-sage_2_0_0-problematic_phosphates')>: None,
 <ScopedKey('Transformation-a5e33f3d8bef56915ccb3a8036c202ba-openff-sage_2_0_0-problematic_phosphates')>: None,
 <ScopedKey('Transformation-ea3bacac1bf318a9aff3a9703edb1fe4-openff-sage_2_0_0-problematic_phosphates')>: None,
 <ScopedKey('Transformation-8d907a3d119265bb194bec7f3f7d0a33-openff-sage_2_0_0-problematic_phosphates')>: None,
 <ScopedKey('Transformation-359ac042fce31f0c810d4c671a5669c2-openff-sage_2_0_0-problematic_phosphates')>

In [39]:
import openfe
alchemical_network = openfe.AlchemicalNetwork.from_json("alchemical_network.json")
[edge for edge in alchemical_network.edges if edge.key == transformation.gufe_key]

/home/joshmitchell/micromamba/envs/alchemiscale-client/lib/python3.12/site-packages/gufe/components/explicitmoleculecomponent.py:93: UserWarning: Partial charges have been provided, these will preferentially be used instead of generating new partial charges
  warnings.warn(wmsg)


[Transformation(stateA=ChemicalSystem(name=COP(=O)(OC)OC | CC(Cl)Cl, components={'solute': SmallMoleculeComponent(name=), 'solvent': ExtendedSolventComponent(name=[H][C]([H])([H])[C]([H])([Cl])[Cl], Na+, Cl-)}), stateB=ChemicalSystem(name=, components={'solvent': ExtendedSolventComponent(name=[H][C]([H])([H])[C]([H])([Cl])[Cl], Na+, Cl-)}), protocol=<ASFEProtocol-1680f0032ebacb09410d687ca0500e05>, name=COP(=O)(OC)OC | CC(Cl)Cl)]

In [55]:
from openff.units import Measurement
for transformation_sk, result in results.items():
    if result is None:
        continue
    edge = [edge for edge in alchemical_network.edges if edge.key == transformation.gufe_key][0]
    print(f"dG({edge.name}) = {result.get_estimate()} +- {result.get_uncertainty()} (n={result.n_protocol_dag_results})")

dG(COP(=O)(OC)OC | CC(Cl)Cl) = -9.737406260846342 kilocalorie_per_mole +- 0.0 kilocalorie_per_mole (n=1)
dG(COP(=O)(OC)OC | CC(Cl)Cl) = -6.531215915657327 kilocalorie_per_mole +- 0.06820387788677357 kilocalorie_per_mole (n=3)
dG(COP(=O)(OC)OC | CC(Cl)Cl) = -8.609516727421038 kilocalorie_per_mole +- 0.0 kilocalorie_per_mole (n=1)
dG(COP(=O)(OC)OC | CC(Cl)Cl) = -6.246417988777921 kilocalorie_per_mole +- 0.03571395572478469 kilocalorie_per_mole (n=3)
dG(COP(=O)(OC)OC | CC(Cl)Cl) = -11.693227045486324 kilocalorie_per_mole +- 0.0 kilocalorie_per_mole (n=1)
dG(COP(=O)(OC)OC | CC(Cl)Cl) = -6.294902972774935 kilocalorie_per_mole +- 0.046813975143571676 kilocalorie_per_mole (n=3)
dG(COP(=O)(OC)OC | CC(Cl)Cl) = -11.15303865685253 kilocalorie_per_mole +- 0.0 kilocalorie_per_mole (n=1)
dG(COP(=O)(OC)OC | CC(Cl)Cl) = -8.012178636512772 kilocalorie_per_mole +- 0.0 kilocalorie_per_mole (n=1)


In [ ]:
alchemical_network.